In [1]:
import os
import time

import tensorflow as tf
import numpy as np

from lib.ops import *
from lib.utils import *

In [2]:
class GAN():
    def __init__(self, sess, epoch, batch_size, z_dim, dataset, model_name, checkpoint_dir, result_dir, logs):
        self._sess = sess
        self._dataset = dataset
        self._model_name = model_name
        self._ckpt_dir = checkpoint_dir
        self._res_dir = result_dir
        self._log_dir = logs
        self._epoch = epoch
        self._batch_size = batch_size

        if self._dataset == 'mnist' or self._dataset == 'fashion-mnist':
            # ground truth input size parameters
            self._input_height = 28
            self._input_width = 28
            self._output_height = 28
            self._output_width = 28

            # dimension of noise-vector
            self._z_dim = z_dim
            self._c_dim = 1

            # for train
            self._lr = 0.0002
            self._beta1 = 0.5

            # save images of generator
            self._sample_counts = 64

            # load dataset for mnist
            self._data_X, self._data_Y = load_mnist(self._dataset)

            # cal batches
            self._batch_nums = len(self._data_X) // self._batch_size
        else:
            raise NotImplementedError

    def discriminator(self, x, is_training=True, reuse=False):
        # build discriminator module
        with tf.variable_scope("discriminator_module", reuse=reuse):
            net = lrelu(conv2d(x, 64, 4, 4, 2, 2, name='discriminator/conv1'))
            net = lrelu(bn(conv2d(net, 128, 4, 4, 2, 2, name='discriminator/conv2'), is_training=is_training, scope='discriminator/bn2'))
            net = tf.reshape(net, [self._batch_size, -1])
            net = lrelu(bn(linear(net, 1024, scope='discriminator/fc3'), is_training=is_training, scope='discriminator/bn3'))
            out_logit = linear(net, 1, scope='discriminator/fc4')
            out = tf.nn.sigmoid(out_logit)
            return out, out_logit, net

    def generator(self, z, is_training=True, reuse=False):
        # build generator module
        with tf.variable_scope("generator_module", reuse=reuse):
            net = tf.nn.relu(bn(linear(z, 1024, scope='generator/fc1'), is_training=is_training, scope='generator/bn1'))
            net = tf.nn.relu(bn(linear(net, 128 * 7 * 7, scope='generator/fc2'), is_training=is_training, scope='generator/bn2'))
            net = tf.reshape(net, [self._batch_size, 7, 7, 128])
            net = tf.nn.relu(
                bn(deconv2d(net, [self._batch_size, 14, 14, 64], 4, 4, 2, 2, name='generator/dc3'), is_training=is_training, scope='generator/bn3'))
            out = tf.nn.sigmoid(deconv2d(net, [self._batch_size, 28, 28, 1], 4, 4, 2, 2, name='generator/dc4'))
            return out

    def build_model(self):

        """ Ground Truth Forward """
        input_dims = [self._input_height, self._input_width, self._c_dim]
        
        self._inputs = tf.placeholder(tf.float32, [self._batch_size] + input_dims, name='real_images')
        D_real, D_real_logits, _ = self.discriminator(self._inputs, is_training=True, reuse=False)
        
        
        """ Random Noises Forward """
        self._z = tf.placeholder(tf.float32, [self._batch_size, self._z_dim], name='z')
        G = self.generator(self._z, is_training=True, reuse=False)
        D_fake, D_fake_logits, _ = self.discriminator(G, is_training=True, reuse=True)
        
        
        """ cal discriminator loss """
        d_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_real_logits, labels=tf.ones_like(D_real)))
        d_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_fake_logits, labels=tf.zeros_like(D_fake)))
        self.D_loss = d_loss_real + d_loss_fake

        
        """ cal generator loss """
        self.G_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_fake_logits, labels=tf.ones_like(D_fake)))

        
        """ Training """
        trainable_variables = tf.trainable_variables()  # collect D, G weights
        d_vars = [var for var in trainable_variables if 'discriminator/' in var.name]
        g_vars = [var for var in trainable_variables if 'generator/' in var.name]
        with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
            self.d_optim = tf.train.AdamOptimizer(self._lr, beta1=self._beta1).minimize(self.D_loss, var_list=d_vars)
            self.g_optim = tf.train.AdamOptimizer(self._lr * 5, beta1=self._beta1).minimize(self.G_loss, var_list=g_vars)


        """" Testing """
        self.generator_images = self.generator(self._z, is_training=False, reuse=True)

    def train(self):
        self.saver = tf.train.Saver()

        # sample noise inputs
        self.sample_z = np.random.uniform(-1, 1, size=(self._batch_size, self._z_dim))

        # loop for epoch
        start_epoch = 0
        start_batch_id = 0
        counter = 1
        start_time = time.time()
        tf.global_variables_initializer().run()
        for epoch in range(start_epoch, self._epoch):
            for idx in range(start_batch_id, self._batch_nums):
                batch_images = self._data_X[idx * self._batch_size:(idx + 1) * self._batch_size]
                batch_z = np.random.uniform(-1, 1, [self._batch_size, self._z_dim]).astype(np.float32)
                # cal d,g loss
                _, d_loss = self._sess.run([self.d_optim, self.D_loss], feed_dict={self._inputs: batch_images, self._z: batch_z})
                _, g_loss = self._sess.run([self.g_optim, self.G_loss], feed_dict={self._z: batch_z})
                # training progress
                counter += 1
                print("Epoch: [%2d] [%4d/%4d] time: %4.4f, d_loss: %.8f, g_loss: %.8f"% (epoch, idx, self._batch_nums, time.time() - start_time, d_loss, g_loss))
                if np.mod(counter, 300) == 0:
                    samples = self._sess.run(self.generator_images, feed_dict={self._z: self.sample_z})
                    tot_num_samples = min(self._sample_counts, self._batch_size)
                    manifold_h = int(np.floor(np.sqrt(tot_num_samples)))
                    manifold_w = int(np.floor(np.sqrt(tot_num_samples)))
                    save_images(samples[:manifold_h * manifold_w, :, :, :], [manifold_h, manifold_w],
                                './' + check_folder(self._res_dir + '/' + self.model_dir) + '/' + self._model_name + '_train_{:02d}_{:04d}.png'.format(epoch, idx))
            start_batch_id = 0
            self.visualize_results(epoch)
        self.save(self._ckpt_dir, counter) # save model

    def visualize_results(self, epoch):
        tot_num_samples = min(self._sample_counts, self._batch_size)
        image_frame_dim = int(np.floor(np.sqrt(tot_num_samples)))
        z_sample = np.random.uniform(-1, 1, size=(self._batch_size, self._z_dim)) # random noise
        samples = self._sess.run(self.generator_images, feed_dict={self._z: z_sample})
        save_images(samples[:image_frame_dim * image_frame_dim, :, :, :], [image_frame_dim, image_frame_dim],
                    check_folder(self._res_dir + '/' + self.model_dir) + '/' + self._model_name + '_epoch%03d' % epoch + '_test_all_classes.png')

    @property
    def model_dir(self):
        return "{}_{}_{}_{}".format(
            self._model_name, self._dataset,
            self._batch_size, self._z_dim)

    def save(self, checkpoint_dir, step):
        checkpoint_dir = os.path.join(checkpoint_dir, self.model_dir, self._model_name)
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        self.saver.save(self._sess, os.path.join(checkpoint_dir, self._model_name + '.model'), global_step=step)

In [3]:
!rm -rf ./checkpoint

with tf.Session(config=tf.ConfigProto(allow_soft_placement=True)) as sess:
    gan = GAN(sess, epoch=2, batch_size=128, z_dim=62, dataset='mnist', model_name='GAN', checkpoint_dir='checkpoint', result_dir='results', logs='logs')
    gan.build_model()
    gan.train()
    print(" [*] Training finished!")
    gan.visualize_results(1)
    print(" [*] Testing finished!")

./data/mnist
reading gz data ./data/mnist/train-images-idx3-ubyte.gz
reading gz data ./data/mnist/train-labels-idx1-ubyte.gz
reading gz data ./data/mnist/t10k-images-idx3-ubyte.gz
reading gz data ./data/mnist/t10k-labels-idx1-ubyte.gz


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch: [ 0] [   0/ 546] time: 2.7983, d_loss: 1.43293023, g_loss: 0.67543036
Epoch: [ 0] [   1/ 546] time: 2.9806, d_loss: 1.44356513, g_loss: 0.68639791
Epoch: [ 0] [   2/ 546] time: 3.3457, d_loss: 1.42303538, g_loss: 0.70779729
Epoch: [ 0] [   3/ 546] time: 3.7443, d_loss: 1.40485501, g_loss: 0.72196198
Epoch: [ 0] [   4/ 546] time: 3.9088, d_loss: 1.39297056, g_loss: 0.73427999
Epoch: [ 0] [   5/ 546] time: 4.0516, d_loss: 1.37945294, g_loss: 0.74004912
Epoch: [ 0] [   6/ 546] time: 4.4346, d_loss: 1.38193369, g_loss: 0.74890316
Epoch: [ 0] [   7/ 546] time: 4.8362, d_loss: 1.37198722, g_loss: 0.75251907
Epoch: [ 0] [   8/ 546] time: 5.0281, d_loss: 1.36477

Epoch: [ 0] [  95/ 546] time: 33.1385, d_loss: 1.13686061, g_loss: 0.91147912
Epoch: [ 0] [  96/ 546] time: 33.4437, d_loss: 1.11716282, g_loss: 0.94564271
Epoch: [ 0] [  97/ 546] time: 33.7353, d_loss: 1.06477809, g_loss: 0.94116604
Epoch: [ 0] [  98/ 546] time: 34.1145, d_loss: 1.08502078, g_loss: 0.98150951
Epoch: [ 0] [  99/ 546] time: 34.4944, d_loss: 1.11710405, g_loss: 1.03580165
Epoch: [ 0] [ 100/ 546] time: 34.8655, d_loss: 1.19773424, g_loss: 0.98668176
Epoch: [ 0] [ 101/ 546] time: 35.1343, d_loss: 1.13615179, g_loss: 0.93394870
Epoch: [ 0] [ 102/ 546] time: 35.3216, d_loss: 1.05479610, g_loss: 0.93448150
Epoch: [ 0] [ 103/ 546] time: 35.6345, d_loss: 1.09842491, g_loss: 0.92244029
Epoch: [ 0] [ 104/ 546] time: 35.9142, d_loss: 1.12946713, g_loss: 0.92257065
Epoch: [ 0] [ 105/ 546] time: 36.2654, d_loss: 1.09022880, g_loss: 0.94227499
Epoch: [ 0] [ 106/ 546] time: 36.6454, d_loss: 1.11064327, g_loss: 0.98819792
Epoch: [ 0] [ 107/ 546] time: 36.9443, d_loss: 1.11311865, g_los

Epoch: [ 0] [ 201/ 546] time: 60.3691, d_loss: 1.32805037, g_loss: 0.90137368
Epoch: [ 0] [ 202/ 546] time: 60.7241, d_loss: 1.32193780, g_loss: 0.88745379
Epoch: [ 0] [ 203/ 546] time: 60.8697, d_loss: 1.27503097, g_loss: 0.81517947
Epoch: [ 0] [ 204/ 546] time: 61.1949, d_loss: 1.24801517, g_loss: 0.77180767
Epoch: [ 0] [ 205/ 546] time: 61.5436, d_loss: 1.29189658, g_loss: 0.80940396
Epoch: [ 0] [ 206/ 546] time: 61.8043, d_loss: 1.26138926, g_loss: 0.81610608
Epoch: [ 0] [ 207/ 546] time: 62.2653, d_loss: 1.25414038, g_loss: 0.84755731
Epoch: [ 0] [ 208/ 546] time: 62.5253, d_loss: 1.28075230, g_loss: 0.87682587
Epoch: [ 0] [ 209/ 546] time: 62.6682, d_loss: 1.30292904, g_loss: 0.87405384
Epoch: [ 0] [ 210/ 546] time: 62.8535, d_loss: 1.28369272, g_loss: 0.80572206
Epoch: [ 0] [ 211/ 546] time: 63.0033, d_loss: 1.27618277, g_loss: 0.83167291
Epoch: [ 0] [ 212/ 546] time: 63.1484, d_loss: 1.28239393, g_loss: 0.83175099
Epoch: [ 0] [ 213/ 546] time: 63.4052, d_loss: 1.27465653, g_los

Epoch: [ 0] [ 307/ 546] time: 84.1247, d_loss: 1.29167604, g_loss: 0.83201313
Epoch: [ 0] [ 308/ 546] time: 84.3868, d_loss: 1.31482601, g_loss: 0.82264996
Epoch: [ 0] [ 309/ 546] time: 84.7749, d_loss: 1.33677387, g_loss: 0.80005407
Epoch: [ 0] [ 310/ 546] time: 84.9330, d_loss: 1.31559384, g_loss: 0.82531047
Epoch: [ 0] [ 311/ 546] time: 85.3251, d_loss: 1.32299125, g_loss: 0.79557073
Epoch: [ 0] [ 312/ 546] time: 85.6033, d_loss: 1.28633702, g_loss: 0.81977099
Epoch: [ 0] [ 313/ 546] time: 85.9448, d_loss: 1.29943132, g_loss: 0.80721176
Epoch: [ 0] [ 314/ 546] time: 86.0878, d_loss: 1.26562703, g_loss: 0.82882762
Epoch: [ 0] [ 315/ 546] time: 86.2441, d_loss: 1.28990078, g_loss: 0.82776260
Epoch: [ 0] [ 316/ 546] time: 86.3853, d_loss: 1.28578579, g_loss: 0.82413357
Epoch: [ 0] [ 317/ 546] time: 86.5305, d_loss: 1.32772183, g_loss: 0.83270681
Epoch: [ 0] [ 318/ 546] time: 86.7848, d_loss: 1.28628469, g_loss: 0.83929873
Epoch: [ 0] [ 319/ 546] time: 86.9347, d_loss: 1.30896485, g_los

Epoch: [ 0] [ 413/ 546] time: 107.1772, d_loss: 1.29864466, g_loss: 0.78535020
Epoch: [ 0] [ 414/ 546] time: 107.3345, d_loss: 1.30063248, g_loss: 0.79647481
Epoch: [ 0] [ 415/ 546] time: 107.6477, d_loss: 1.30840659, g_loss: 0.81005931
Epoch: [ 0] [ 416/ 546] time: 107.8654, d_loss: 1.30942678, g_loss: 0.82947809
Epoch: [ 0] [ 417/ 546] time: 108.1996, d_loss: 1.30432165, g_loss: 0.82327360
Epoch: [ 0] [ 418/ 546] time: 108.4195, d_loss: 1.31813908, g_loss: 0.77914488
Epoch: [ 0] [ 419/ 546] time: 108.5823, d_loss: 1.31388628, g_loss: 0.78481340
Epoch: [ 0] [ 420/ 546] time: 108.7289, d_loss: 1.30319047, g_loss: 0.79765451
Epoch: [ 0] [ 421/ 546] time: 108.8773, d_loss: 1.33061242, g_loss: 0.80855834
Epoch: [ 0] [ 422/ 546] time: 109.1556, d_loss: 1.31077921, g_loss: 0.80897045
Epoch: [ 0] [ 423/ 546] time: 109.4207, d_loss: 1.31332266, g_loss: 0.78349435
Epoch: [ 0] [ 424/ 546] time: 109.6367, d_loss: 1.29836822, g_loss: 0.80598164
Epoch: [ 0] [ 425/ 546] time: 109.8986, d_loss: 1.31

...

Epoch: [ 1] [  76/ 546] time: 157.1534, d_loss: 1.32214355, g_loss: 0.80945098
Epoch: [ 1] [  77/ 546] time: 157.3088, d_loss: 1.34421444, g_loss: 0.78746307
Epoch: [ 1] [  78/ 546] time: 157.6551, d_loss: 1.31395006, g_loss: 0.78556669
Epoch: [ 1] [  79/ 546] time: 157.9253, d_loss: 1.32463050, g_loss: 0.79009140
Epoch: [ 1] [  80/ 546] time: 158.1614, d_loss: 1.30339515, g_loss: 0.79997301
Epoch: [ 1] [  81/ 546] time: 158.4128, d_loss: 1.28765988, g_loss: 0.79129773
Epoch: [ 1] [  82/ 546] time: 158.5739, d_loss: 1.27268219, g_loss: 0.81150037
Epoch: [ 1] [  83/ 546] time: 158.7194, d_loss: 1.29181194, g_loss: 0.80459839
Epoch: [ 1] [  84/ 546] time: 158.8630, d_loss: 1.32146096, g_loss: 0.78323472
Epoch: [ 1] [  85/ 546] time: 159.0092, d_loss: 1.30458260, g_loss: 0.80020940
Epoch: [ 1] [  86/ 546] time: 159.2368, d_loss: 1.31742191, g_loss: 0.80602723
Epoch: [ 1] [  87/ 546] time: 159.5487, d_loss: 1.33340478, g_loss: 0.80530345
Epoch: [ 1] [  88/ 546] time: 159.7593, d_loss: 1.32

Epoch: [ 1] [ 180/ 546] time: 181.1921, d_loss: 1.33741856, g_loss: 0.77983266
Epoch: [ 1] [ 181/ 546] time: 181.3528, d_loss: 1.29207993, g_loss: 0.79133135
Epoch: [ 1] [ 182/ 546] time: 181.5075, d_loss: 1.30238438, g_loss: 0.81084263
Epoch: [ 1] [ 183/ 546] time: 181.8262, d_loss: 1.31564474, g_loss: 0.79775393
Epoch: [ 1] [ 184/ 546] time: 182.1252, d_loss: 1.30355763, g_loss: 0.79789233
Epoch: [ 1] [ 185/ 546] time: 182.2898, d_loss: 1.30928588, g_loss: 0.79884982
Epoch: [ 1] [ 186/ 546] time: 182.5754, d_loss: 1.30902576, g_loss: 0.80267382
Epoch: [ 1] [ 187/ 546] time: 182.9144, d_loss: 1.31575918, g_loss: 0.79006851
Epoch: [ 1] [ 188/ 546] time: 183.1319, d_loss: 1.30383849, g_loss: 0.80176687
Epoch: [ 1] [ 189/ 546] time: 183.4163, d_loss: 1.30320585, g_loss: 0.79777592
Epoch: [ 1] [ 190/ 546] time: 183.5629, d_loss: 1.32521355, g_loss: 0.77848756
Epoch: [ 1] [ 191/ 546] time: 183.7879, d_loss: 1.29128206, g_loss: 0.81596321
Epoch: [ 1] [ 192/ 546] time: 183.9397, d_loss: 1.33

Epoch: [ 1] [ 284/ 546] time: 203.4838, d_loss: 1.33679712, g_loss: 0.78539801
Epoch: [ 1] [ 285/ 546] time: 203.6793, d_loss: 1.31784368, g_loss: 0.80787647
Epoch: [ 1] [ 286/ 546] time: 203.9648, d_loss: 1.30990171, g_loss: 0.79178685
Epoch: [ 1] [ 287/ 546] time: 204.3146, d_loss: 1.31810462, g_loss: 0.80080128
Epoch: [ 1] [ 288/ 546] time: 204.4611, d_loss: 1.33474946, g_loss: 0.77986079
Epoch: [ 1] [ 289/ 546] time: 204.6127, d_loss: 1.33275056, g_loss: 0.78849709
Epoch: [ 1] [ 290/ 546] time: 204.7559, d_loss: 1.34672451, g_loss: 0.77443898
Epoch: [ 1] [ 291/ 546] time: 204.9009, d_loss: 1.33525240, g_loss: 0.78458863
Epoch: [ 1] [ 292/ 546] time: 205.1438, d_loss: 1.32812452, g_loss: 0.77692950
Epoch: [ 1] [ 293/ 546] time: 205.5257, d_loss: 1.33102298, g_loss: 0.79821157
Epoch: [ 1] [ 294/ 546] time: 205.7945, d_loss: 1.31295121, g_loss: 0.79641682
Epoch: [ 1] [ 295/ 546] time: 205.9391, d_loss: 1.34519267, g_loss: 0.80502158
Epoch: [ 1] [ 296/ 546] time: 206.0853, d_loss: 1.33

Epoch: [ 1] [ 388/ 546] time: 226.8285, d_loss: 1.32958174, g_loss: 0.81816399
Epoch: [ 1] [ 389/ 546] time: 227.2057, d_loss: 1.32512891, g_loss: 0.82957864
Epoch: [ 1] [ 390/ 546] time: 227.5803, d_loss: 1.35751510, g_loss: 0.80920148
Epoch: [ 1] [ 391/ 546] time: 227.8198, d_loss: 1.36524391, g_loss: 0.79678673
Epoch: [ 1] [ 392/ 546] time: 228.1445, d_loss: 1.31096268, g_loss: 0.79953575
Epoch: [ 1] [ 393/ 546] time: 228.3787, d_loss: 1.33061922, g_loss: 0.80640769
Epoch: [ 1] [ 394/ 546] time: 228.5917, d_loss: 1.34997916, g_loss: 0.79513836
Epoch: [ 1] [ 395/ 546] time: 228.8308, d_loss: 1.37073922, g_loss: 0.78713572
Epoch: [ 1] [ 396/ 546] time: 229.0595, d_loss: 1.31374204, g_loss: 0.81198615
Epoch: [ 1] [ 397/ 546] time: 229.4353, d_loss: 1.34220624, g_loss: 0.77956319
Epoch: [ 1] [ 398/ 546] time: 229.7857, d_loss: 1.32450652, g_loss: 0.78217500
Epoch: [ 1] [ 399/ 546] time: 230.1693, d_loss: 1.32423019, g_loss: 0.79701906
Epoch: [ 1] [ 400/ 546] time: 230.4680, d_loss: 1.32

Epoch: [ 1] [ 493/ 546] time: 252.2141, d_loss: 1.29719472, g_loss: 0.82642412
Epoch: [ 1] [ 494/ 546] time: 252.5153, d_loss: 1.31901360, g_loss: 0.81567383
Epoch: [ 1] [ 495/ 546] time: 252.8350, d_loss: 1.33044720, g_loss: 0.79983598
Epoch: [ 1] [ 496/ 546] time: 253.2148, d_loss: 1.28169203, g_loss: 0.80350196
Epoch: [ 1] [ 497/ 546] time: 253.5042, d_loss: 1.33588123, g_loss: 0.75300294
Epoch: [ 1] [ 498/ 546] time: 253.6933, d_loss: 1.32531929, g_loss: 0.77787280
Epoch: [ 1] [ 499/ 546] time: 253.9464, d_loss: 1.30388963, g_loss: 0.78283489
Epoch: [ 1] [ 500/ 546] time: 254.1376, d_loss: 1.32253361, g_loss: 0.80325580
Epoch: [ 1] [ 501/ 546] time: 254.4739, d_loss: 1.30432928, g_loss: 0.82036376
Epoch: [ 1] [ 502/ 546] time: 254.7051, d_loss: 1.32558012, g_loss: 0.80440074
Epoch: [ 1] [ 503/ 546] time: 254.8489, d_loss: 1.30410409, g_loss: 0.81467164
Epoch: [ 1] [ 504/ 546] time: 255.0571, d_loss: 1.31433702, g_loss: 0.80217940
Epoch: [ 1] [ 505/ 546] time: 255.3644, d_loss: 1.30